# Lecture 1: Breaking RSA with SPA - Introduction

## RSA Cryptosystem

RSA (Rivest–Shamir–Adleman) is a public-key cryptosystem that is widely used for secure data transmission. It is also one of the oldest. The acronym RSA comes from the surnames of Ron Rivest, Adi Shamir, and Leonard Adleman, who publicly described the algorithm in 1977. An equivalent system was developed secretly, in 1973 at GCHQ (the British signals intelligence agency), by the English mathematician Clifford Cocks. That system was declassified in 1997. (from https://en.wikipedia.org/wiki/RSA_(cryptosystem))

### How it works

#### Parameters

The parameters defining a RSA cryptosystem are given by a few integers:
* a prime $p$, a prime $q$, the so called _modulus_ $n = p \cdot q$,
* a _public exponent_ $e$ with $\mathrm{gcd}(e, \phi(n)) = 1$,
* a _private exponent_ $d$ with $d \cdot e \equiv 1 \mod \phi(n)$.

In real applications $n$ has a bit-length of 2k to 4k and $e$ is often set to the 4th Fermat number $F_4 = 2^{2^4} = 65537 = 0x10001$.

#### Encryption

Given a message $m \in [0..n]$ the _encryption_ function is given by:
$$ c := m^e \mod n,$$
where $c$ is the resulting ciphertext.

#### Decryption
Given a ciphertext $c \in [0..n]$ the _decryption_ fcuntion is given by:

$$ m := c^d \mod n.$$

#### Proof
Fermat's little theorem ;-)

## How to do in C?

In C it's not obvious how to do modular exponentiation. But a well-known algorithm called _Square-and-Multiply_ helps:

```
// Calculate x^k
// b: Binary representation of k
// res: Result

function bin_exp(x,b)
  res = 1
  for i = n..0
    res = res^2
    if b_i == 1
      res = res * x
    end-if
  end-for
  return res
end-function
```
(Pseudocode from https://de.wikipedia.org/wiki/Bin%C3%A4re_Exponentiation)

## Capture and Attack!

### Implementation

This leads us to the following concrete implementation in C where we used only integers of size `uint8_t`:
```c
uint8_t exponent = private_exponent;
uint8_t message = 0xA0;

uint16_t tmp;
uint8_t result = 1;
while (exponent)
{
    if (exponent & 1)
    {
        tmp = result * message;
        result = tmp % modulus;
    }

    tmp = message * message;
    message = tmp % modulus;
    exponent >>= 1;
}
```

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 1</div>
    
1. Explain why the above code works. 
2. Explain why `tmp` is of type `uint16_t`.

</div>

### Record a trace

In [22]:
import securec
import securec.util as util
scope, target = util.init()

See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


In [23]:
securec.util.compile_and_flash('./1_rsa_uint8_fixed.c')

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2177 bytes
✓


In [29]:
import struct
import time
import warnings
    
scope.default_setup()
scope.adc.samples = 10000

def capture():
    scope.arm()
    target.simpleserial_write('r', b'')
    return util.capture()

In [30]:
trace = capture()

In [31]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10

output_notebook()

Loading BokehJS ...

In [32]:
p = figure(width=900, height=800)
p.add_tools(CrosshairTool())
p.line(range(0, len(trace)), trace)
show(p)

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 2</div>

1. Try to explain the picture above! What do you see? Can you tell the exponent? If not, have a look into the code. Do you "see" the exponent now?
2. Make familiar with the capture code. You'll need it often...

</div>

In [16]:
util.exit()

In [21]:
(0xa0 ** 5) % 221

62